# Dev

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
sys.path.insert(0, f"{os.getcwd()}/src")

In [3]:
from pprint import pprint

In [5]:
from transformers.tokenization_fsmt import FSMTTokenizer

# fairseq tokenizer

hub_utils.py:
        
    def encode(self, sentence: str) -> torch.LongTensor:
        sentence = self.tokenize(sentence)
        sentence = self.apply_bpe(sentence)
        return self.binarize(sentence)

    def decode(self, tokens: torch.LongTensor) -> str:
        sentence = self.string(tokens)
        sentence = self.remove_bpe(sentence)
        return self.detokenize(sentence)

    def tokenize(self, sentence: str) -> str:
        if self.tokenizer is not None:
            sentence = self.tokenizer.encode(sentence)
        return sentence

    def detokenize(self, sentence: str) -> str:
        if self.tokenizer is not None:
            sentence = self.tokenizer.decode(sentence)
        return sentence

    def apply_bpe(self, sentence: str) -> str:
        if self.bpe is not None:
            sentence = self.bpe.encode(sentence)
        return sentence

    def remove_bpe(self, sentence: str) -> str:
        if self.bpe is not None:
            sentence = self.bpe.decode(sentence)
        return sentence

    def binarize(self, sentence: str) -> torch.LongTensor:
        return self.src_dict.encode_line(sentence, add_if_not_exist=False).long()

    def string(self, tokens: torch.LongTensor) -> str:
        return self.tgt_dict.string(tokens)       
        
1. tokenize using tokenizer.perl from mosesdecoder https://github.com/moses-smt/mosesdecoder



2. apply_bpe.py script using the wmt14.en-fr.fconv-cuda/bpecodes file
https://github.com/rsennrich/subword-nmt/blob/master/subword_nmt/apply_bpe.py

fairseq/data/encoders/fastbpe.py uses fastBPE
loads "model/bpecodes" file


3. binarize

somehow need to re-use bpe codes:https://github.com/facebookresearch/XLM#download--preprocess-data

> If you want to use our pretrained models, you need to have an exactly identical vocabulary. Since small differences can happen during preprocessing, we recommend that you use our BPE codes and vocabulary (although you should get something almost identical if you learn the codes and compute the vocabulary yourself).


# fastBPE

In [6]:
line = "Machine Learning"

In [7]:
import fastBPE
import fairseq
bpe_codes = 'data/wmt19.ru-en.ensemble/bpecodes'
codes = fairseq.file_utils.cached_path(bpe_codes)
codes
bpe = fastBPE.fastBPE(codes)
bpe
bpe_symbol = "@@ "
def encode(x: str) -> str: return bpe.apply([x])[0]
def decode(x: str) -> str: return (x + ' ').replace(bpe_symbol, '').rstrip()
encoded = encode("Machine Learning")
encoded
decoded = decode(encoded)
decoded

'data/wmt19.ru-en.ensemble/bpecodes'

'Mach@@ ine L@@ ear@@ ning'

'Machine Learning'

In [8]:
from transformers import XLMTokenizer, CTRLTokenizer
tokenizer = XLMTokenizer.from_pretrained("xlm-mlm-100-1280")
tokenizer.tokenize(line)
tokenizer = CTRLTokenizer.from_pretrained("xlm-mlm-100-1280")
tokenizer.tokenize(line)


['Machine</w>', 'Le', 'arning</w>']

['M@@', 'a@@', 'c@@', 'h@@', 'i@@', 'n@@', 'e', 'L@@', 'e@@', 'a@@', 'r@@', 'n@@', 'i@@', 'n@@', 'g']

In [10]:
#tokenizer = FSMTTokenizer.from_pretrained('data/wmt19.ru-en.ensemble/')

OSError: Model name 'data/wmt19.ru-en.ensemble/' was not found in tokenizers model name list (fsmt-wmt19-ru-en). We assumed 'data/wmt19.ru-en.ensemble/' was a path, a model identifier, or url to a directory containing vocabulary files named ['vocab.json', 'merges.txt'] but couldn't find such vocabulary files at this path or url.

In [11]:
import inspect, os
os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))

'/mnt/nvme1/code/huggingface/transformers-fair-wmt'

# Explore checkpoint's contents

In [12]:
import torch
path = "/code/huggingface/transformers-fair-wmt"
chkpt1_path = path + "/data/wmt19.ru-en.ensemble/model4.pt"

In [13]:
chkpt = torch.load(chkpt1_path, map_location="cpu")

In [14]:
chkpt.keys()

dict_keys(['args', 'model', 'optimizer_history', 'last_optimizer_state', 'extra_state'])

In [15]:
from pprint import pprint


In [16]:
print(1)

1


In [17]:
pprint(vars(chkpt["args"]))

{'adam_betas': '(0.9, 0.98)',
 'adam_eps': 1e-08,
 'adaptive_input': False,
 'adaptive_softmax_cutoff': None,
 'adaptive_softmax_dropout': 0,
 'arch': 'transformer_wmt_en_de_big',
 'attention_dropout': 0.1,
 'bucket_cap_mb': 25,
 'clip_norm': 0.0,
 'cpu': False,
 'criterion': 'label_smoothed_cross_entropy',
 'data': ['/private/home/edunov/wmt19/data/old/enru',
          '/private/home/edunov/wmt19/data/old/enru',
          '/private/home/edunov/wmt19/data/finetune/ncru/',
          '/private/home/edunov/wmt19/data/temp/enru'],
 'ddp_backend': 'c10d',
 'decoder_attention_heads': 16,
 'decoder_embed_dim': 1024,
 'decoder_embed_path': None,
 'decoder_ffn_embed_dim': 4096,
 'decoder_input_dim': 1024,
 'decoder_layers': 6,
 'decoder_learned_pos': False,
 'decoder_normalize_before': False,
 'decoder_output_dim': 1024,
 'device_id': 0,
 'distributed_backend': 'nccl',
 'distributed_init_method': 'tcp://localhost:17185',
 'distributed_port': -1,
 'distributed_rank': 0,
 'distributed_world_size'

In [18]:
pprint(chkpt["optimizer_history"])

[{'criterion_name': 'LabelSmoothedCrossEntropyCriterion',
  'lr_scheduler_state': {'best': 3.449914911560068},
  'num_updates': 200000,
  'optimizer_name': 'FP16Optimizer'},
 {'criterion_name': 'LabelSmoothedCrossEntropyCriterion',
  'lr_scheduler_state': {'best': 3.449914911560068},
  'num_updates': 201600,
  'optimizer_name': 'FP16Optimizer'}]


In [19]:
pprint(chkpt["last_optimizer_state"])

OrderedDict([('state',
              OrderedDict([(139681734876832,
                            OrderedDict([('step', 201600),
                                         ('exp_avg',
                                          tensor([ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -1.3744e-04,
        -1.2752e-04, -2.1968e-05])),
                                         ('exp_avg_sq',
                                          tensor([0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 3.6497e-07, 5.2216e-07,
        5.1033e-07]))]))])),
             ('param_groups',
              [OrderedDict([('lr', 9.860132971832693e-05),
                            ('betas', (0.9, 0.98)),
                            ('eps', 1e-08),
                            ('weight_decay', 0.0),
                            ('amsgrad', False),
                            ('params', [139681734876832])])]),
             ('loss_scale', 1.0)])


In [20]:
pprint(chkpt["extra_state"])

{'best': 2.870054022029137,
 'train_iterator': {'epoch': 13, 'iterations_in_epoch': 1601},
 'train_meters': OrderedDict([('train_loss',
                               <fairseq.logging.meters.AverageMeter object at 0x7fbd014ad150>),
                              ('train_nll_loss',
                               <fairseq.logging.meters.AverageMeter object at 0x7fbd014ad310>),
                              ('valid_loss',
                               <fairseq.logging.meters.AverageMeter object at 0x7fbd014ad390>),
                              ('valid_nll_loss',
                               <fairseq.logging.meters.AverageMeter object at 0x7fbd014ad410>),
                              ('wps',
                               <fairseq.logging.meters.TimeMeter object at 0x7fbd014ad490>),
                              ('ups',
                               <fairseq.logging.meters.TimeMeter object at 0x7fbd014ad5d0>),
                              ('wpb',
                               <fairs

# Load model via fairseq

In [21]:
from fairseq import hub_utils
checkpoint_file = 'model1.pt:model2.pt:model3.pt:model4.pt'
model_name_or_path = 'transformer.wmt19.ru-en'
data_name_or_path = '.'
cls = fairseq.model_parallel.models.transformer.ModelParallelTransformerModel
models = cls.hub_models()
kwargs = {'bpe': 'fastbpe', 'tokenizer': 'moses'}
x = hub_utils.from_pretrained(
            model_name_or_path,
            checkpoint_file,
            data_name_or_path,
            archive_map=models,
            **kwargs
        )

In [22]:
pprint(vars(x["args"]))

{'activation_dropout': 0.0,
 'activation_fn': 'relu',
 'adam_betas': '(0.9, 0.98)',
 'adam_eps': 1e-08,
 'adaptive_input': False,
 'adaptive_softmax_cutoff': None,
 'adaptive_softmax_dropout': 0,
 'arch': 'transformer_wmt_en_de_big',
 'attention_dropout': 0.1,
 'bpe': 'fastbpe',
 'bpe_codes': '/home/stas/.cache/torch/pytorch_fairseq/1f635f61a93197b2be015bcdc60f47829db1172b5b81547d72b7b28235b28fa9.5e1a2ea19b51d6733a96ed885e05dc2d3d1e3cb3573b9e62ee7acfb0454ee976/bpecodes',
 'bucket_cap_mb': 25,
 'clip_norm': 0.0,
 'cpu': False,
 'criterion': 'label_smoothed_cross_entropy',
 'cross_self_attention': False,
 'data': '/home/stas/.cache/torch/pytorch_fairseq/1f635f61a93197b2be015bcdc60f47829db1172b5b81547d72b7b28235b28fa9.5e1a2ea19b51d6733a96ed885e05dc2d3d1e3cb3573b9e62ee7acfb0454ee976',
 'ddp_backend': 'c10d',
 'decoder_attention_heads': 16,
 'decoder_embed_dim': 1024,
 'decoder_embed_path': None,
 'decoder_ffn_embed_dim': 4096,
 'decoder_input_dim': 1024,
 'decoder_layerdrop': 0,
 'decoder_

In [23]:
pprint(x["task"])

In [24]:
pprint(x["models"]) # 4 models

[TransformerModel(
  (encoder): TransformerEncoder(
    (dropout_module): FairseqDropout()
    (embed_tokens): Embedding(31232, 1024, padding_idx=1)
    (embed_positions): SinusoidalPositionalEmbedding()
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (dropout_module): FairseqDropout()
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (dropout_module): FairseqDropout()
        (activation_dropout_module): FairseqDropout()
        (fc1): Linear(in_features=1024, out_features=8192, bias=True)
        (fc2): Linear(in_features=8192, out_features=1024, bias=True)
        (fin

# Encode Process

## Tokenize

In [25]:
args = x["args"] # loaded from the checkpoint earlier
from fairseq.data.encoders.moses_tokenizer import MosesTokenizer
#from argparse import Namespace
#args = argparse.Namespace(moses_source_lang='ru_RU', moses_target_lang='en_XX')
# fairseq.data.encoders.moses_tokenizer.MosesTokenizer
sentence = "Машинное обучение - это здорово!"
tokenizer = MosesTokenizer(args)
sentence = tokenizer.encode(sentence)
sentence

# well it mainly just adds whitespace where needed

'Машинное обучение - это здорово !'

## Apply BPE

In [26]:
from fairseq.data.encoders.fastbpe import fastBPE
bpe = fastBPE(args)
sentence = bpe.encode(sentence)
sentence


'Ма@@ шин@@ ное обучение - это здорово !'

## Binarize (bytecodes into numbers)

In [27]:
# %%pixie_debugger -b /mnt/nvme1/code/github/00nlp/fairseq/fairseq/data/dictionary.py:30

from fairseq.data.dictionary import Dictionary

filename = '/home/stas/.cache/torch/pytorch_fairseq/1f635f61a93197b2be015bcdc60f47829db1172b5b81547d72b7b28235b28fa9.5e1a2ea19b51d6733a96ed885e05dc2d3d1e3cb3573b9e62ee7acfb0454ee976/dict.ru.txt'
#cls = fairseq.tasks.translation.TranslationTask

#src_dict = Dictionary(args)
src_dict = Dictionary.load(filename)
#dir(src_dict)

# self.binarize(sentence) equivalent
src_dict.encode_line(sentence, add_if_not_exist=False).long()
# setup_task loads both dicts

# this looks up the tokenized bpe strings in the dict and replaces them with the dict indices
src_dict.index("Ма@@")
src_dict.index("обучение@@") # unk (this token doesn't exist in the dict)

# the vocab is this dict:
# src_dict.indices
# can be fed directly to json.dumps => vocab.json

tensor([  648, 13440,    97,  3618,    25,    74, 22548,   384,     2])

648

3

In [28]:
src_dict.indices.keys()

dict_keys(['<s>', '<pad>', '</s>', '<unk>', ',', '.', 'и', 'в', 'на', ')', ':', 'по', 'с', '/', '@-@', '(', '&quot;', 'для', 'что', 'о', 'не', 'к', 'В', 'года', ';', '-', 'или', 'а', 'также', 'от', 'за', 'из', '*', 'как', '1', 'Наций', 'Объединенных', 'Организации', '&gt;', 'том', 'их', '&lt;', '2', 'его', 'которые', 'не@@', '3', 'чтобы', 'области', 'того', '10', 'ные', 'ных', 'при', '4', 'развития', 'человека', 'то', 'будет', 'ной', 'до', 'но', 'более', '5', 'я', 'году', 'ного', 'отношении', 'этой', 'связи', 'ным', '#', 'деятельности', 'вопросам', 'это', 'ее', '12', 'бы', 'прав', 'все', 'всех', 'является', '6', 'было', '00', 'во', 'соответствии', 'ли', 'были', 'об', 'н', '11', 'женщин', 'время', '7', 'между', 'ный', 'ное', 'может', 'рамках', '?', '15', 'a', 'быть', 'тем', 'т', 'ными', 'стран', 'со', 'мы', '8', '2@@', 'он', 'у@@', 'целях', '9', 'резолюции', 'работы', 'этого', 'у', 'Комитет', 'A', 'ном', 'сессии', '20', 'С@@', 'был', '14', 'США', '3@@', 'системы', 'они', 'ная', 'та', 'д

In [29]:
import re

In [30]:
x = "\u8de8@@"
re.sub(r'@@', '</w>', x)

'跨</w>'

In [31]:
d = {'le@@': 5, 'tt@@': 6, 'er': 7}
d

{'le@@': 5, 'tt@@': 6, 'er': 7}

In [32]:
d2 = dict((re.sub(r'@@', '', k), v) if k.endswith('@@') else (re.sub(r'$', '</w>', k), v) for k, v in d.items())
d2

{'le': 5, 'tt': 6, 'er</w>': 7}

In [33]:
d1 = {'a': 1, 'b': 2, 'c': 3}
d2 = d1.copy()
d2["a"]=5
ks = ['a', 'c']
for k in ks: d2[k] = d1[k]
d2

{'a': 1, 'b': 2, 'c': 3}

In [34]:
merges = ["a b 3993", "a c 93939", "d k 3"]
[ a for a,b,c in merge.split()[:2] for merge in merges]

NameError: name 'merge' is not defined

In [ ]:
d = {'le@@': 5, 'tt@@': 6, 'er': 7}
ks = ['le@@', 'er']
#d1[]
len(d)